# Polarization Curve Analytical Modeling

For the analytical modeling, the following equation was considered:

$$
E(i) = E_0 - b \cdot \ln \left[ \frac{1}{2} \left( \frac{i}{i_0} + \sqrt{\left(\frac{i}{i_0}\right)^2 + 4} \right) \right] - R \cdot i + \frac{1}{i - i_L}
$$

where $i_L$ and $i_0$ are the limiting and exchange current densities, respectively, $R$ is the system's ohmic resistance, $E_0$ is the equilibrium potential, and $b$ is the Tafel slope. 

This equation expresses the voltage losses in terms of overpotential starting from the equilibrium potential. In order, the terms represent: 

- **Kinetic or activation loss:** described by the Butler-Volmer equation (or Tafel form in this case),
- **Ohmic loss:** due to material resistance,
- **Mass transport loss:** due to the limited supply of reactants at the electrode surface, leading to concentration gradients in the electrolyte or near the electrode interface. 


# Library import

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# Creation of simulated curves

Choose number of values per parameter and number of points per curve

In [21]:
n_val = 12

n_points = 40

In [ ]:
E0_min = 1.2631     #V
E0_max = 1.5056     #V

b_min = 28.78       #mV/dec
b_max = 88          #mV/dec

i0_min = 0.38       # μA/cm²
i0_max = 705.22     # μA/cm²

R_min = 0.69        #Ω·cm²
R_max = 21.14       #Ω·cm²

iL_min = 42.117     #mA/cm²
iL_max = 570.699    #mA/cm²

# Rescale for the function --> all the terms in [V]
E0_min_real = E0_min    #V
E0_max_real = E0_max    #V

b_min_real = b_min / (1000 * np.log(10))    #V
b_max_real = b_max / (1000 * np.log(10))    #V

i0_min_real = i0_min / 1000     #mA/cm²
i0_max_real = i0_max / 1000     #mA/cm²

R_min_real = R_min / 1000       #kΩ·cm²
R_max_real = R_max / 1000       #kΩ·cm²

iL_min_real = iL_min    #mA/cm²
iL_max_real = iL_max    #mA/cm²

E0_range = np.linspace(E0_min_real, E0_max_real, n_val)
b_range = np.linspace(b_min_real, b_max_real, n_val)
i0_range = np.logspace(np.log10(i0_min_real), np.log10(i0_max_real), n_val)
R_range = np.logspace(np.log10(R_min_real), np.log10(R_max_real), n_val)
iL_range = np.logspace(np.log10(iL_min), np.log10(iL_max), n_val)
file_name = 'params'

n_params = len(E0_range) * len(b_range) * len(i0_range) * len(R_range) * len(iL_range)
curves = np.full((n_params, n_points, 2), np.nan)
params = np.full((n_params, 5), np.nan)

Define the analytic function of the polarization curves

In [23]:
def E(i, E0, b, i0, R, iL):
    """
    Calculate E(i) based on the given parameters with specific units.

    Parameters:
    - i: Current density (mA/cm²)
    - E0: Standard potential (V)
    - b: Tafel slope (V)
    - i0: Exchange current density (mA/cm²)
    - R: Resistance (kOhm·cm²)
    - iL: Limiting current density (mA/cm²)

    Returns:
    - E: The calculated potential (V)
    """
    
    E_tot = E0 - b * np.log(0.5 * ((i / i0) + np.sqrt((i / i0)**2 + 4))) - R * i + 1 / (i - iL)

    return E_tot

Curves creation and saving

In [ ]:
idx = 0

curves = np.full((len(E0_range) * len(b_range) * len(i0_range) * len(R_range) * len(iL_range), n_points, 2), np.nan)
params = np.full((curves.shape[0], 5), np.nan)

for E0 in E0_range:
    for b in b_range:
        for i0 in i0_range:
            for R in R_range:
                for iL in iL_range:

                    # Initial i_max (93% of iL)
                    i_max = 0.93 * iL  

                    # Construction of current points
                    if i_max <= 20:
                        # Only range 0–i_max with step 1
                        i_values = np.arange(0, i_max + 1, 1)
                    else:
                        # First range: 0–20 with step 1 (21 points)
                        i_dense = np.arange(0, 21, 1)

                        # Number of remaining points to complete n_points
                        n_sparse = n_points - len(i_dense)

                        # Second interval: from 20 (excluded) to i_max
                        if n_sparse > 0:
                            i_sparse = np.linspace(20, i_max, n_sparse + 1, endpoint=True)[1:]  # exclude 20
                            i_values = np.concatenate((i_dense, i_sparse))
                        else:
                            i_values = i_dense  # in case n_points <= 21

                    # Initial curve computation
                    E_values = E(i_values, E0, b, i0, R, iL)

                    # If the curve drops below 0.2 V → find i such that E(i) = 0.2
                    if np.any(E_values < 0.2):
                        E_func = lambda i: E(i, E0, b, i0, R, iL) - 0.2
                        initial_guess = iL - 1e-4
                        try:
                            i_target = fsolve(E_func, initial_guess)[0]
                            i_max = i_target
                        except Exception as e:
                            print(f"Warning: i for E=0.2 not found, using original curve (idx={idx})")

                        # Rebuild point distribution using the same logic
                        if i_max <= 20:
                            i_values = np.arange(0, i_max + 1, 1)
                        else:
                            i_dense = np.arange(0, 21, 1)
                            n_sparse = n_points - len(i_dense)
                            if n_sparse > 0:
                                i_sparse = np.linspace(20, i_max, n_sparse + 1, endpoint=True)[1:]
                                i_values = np.concatenate((i_dense, i_sparse))
                            else:
                                i_values = i_dense

                        # Recompute the curve up to E = 0.2
                        E_values = E(i_values, E0, b, i0, R, iL)

                    # Final storage
                    curves[idx, :, 0] = i_values
                    curves[idx, :, 1] = E_values
                    params[idx, :] = [E0, b, i0, R, iL]

                    idx += 1

In [25]:
# Saving
np.save(f'curves_{n_val}.npy', curves)
np.save(f'params_{n_val}.npy', params)

## Dataset separation

In [ ]:
# Tolerance for separation into iL-dataset and no-iL-dataset
tol = 5e-5

Derivative computation

In [26]:
derivatives = []
derivatives_norm = []

for idx in range(n_params):
    c = curves[idx]
    
    # Avoid possible NaNs
    mask_valid = ~np.isnan(c[:,0]) & ~np.isnan(c[:,1])
    i_vals = c[mask_valid, 0]
    E_vals = c[mask_valid, 1]
    
    # Derivative using gradient → same length as i_vals
    dE_di = np.gradient(E_vals, i_vals)
    derivatives.append((i_vals, dE_di))
    
    # Current normalization between 0 and 1
    i_norm = (i_vals - i_vals.min()) / (i_vals.max() - i_vals.min())
    derivatives_norm.append((i_norm, dE_di))

Separation through derivative values comparison

In [ ]:
# Comparison between derivative values
mean_second_list = []
mean_last_list = []

for i_vals, dE_di in derivatives:
    n_points = len(dE_di)
    
    # Mean over the 33%–66% block
    start_second = int(n_points * 0.33)
    end_second   = int(n_points * 0.66)
    mean_second = np.mean(dE_di[start_second:end_second])
    
    # Mean over the last 3 points
    mean_last = np.mean(dE_di[-2:])
    
    mean_second_list.append(mean_second)
    mean_last_list.append(mean_last)

dataset_with_iL = []
params_with_iL = []
dataset_without_iL = []
params_without_iL = []

for idx in range(n_params):
    c = curves[idx]
    mean_second = mean_second_list[idx]
    mean_last   = mean_last_list[idx]
    
    if abs(abs(mean_last) - abs(mean_second)) > tol:
        if abs(mean_last) > abs(mean_second):
            dataset_with_iL.append(c)
            params_with_iL.append(params[idx])
        else:
            dataset_without_iL.append(c)
            params_without_iL.append(params[idx])
    else:
        dataset_without_iL.append(c)
        params_without_iL.append(params[idx])

dataset_con_iL = np.array(dataset_with_iL, dtype=float)
params_con_iL = np.array(params_with_iL, dtype=float)

dataset_senza_iL = np.array(dataset_without_iL, dtype=float)
params_senza_iL = np.array(params_without_iL, dtype=float)

print(f"Number of curves with iL: {len(dataset_with_iL)}")
print(f"Number of curves without iL: {len(dataset_without_iL)}")

## Saving the different datasets

In [29]:
np.save(f'curves_with_iL_{n_val}.npy', dataset_with_iL)
np.save(f'params_with_iL_{n_val}.npy', params_with_iL)
np.save(f'curves_without_iL_{n_val}.npy', dataset_without_iL)
np.save(f'params_without_iL_{n_val}.npy', params_without_iL)